In [3]:
from os import listdir,getcwd
from os.path import isfile, join
taggedText = []
text = []
filepath = join(getcwd(), "browncopy")
allFiles = [f for f in listdir(filepath) if not (f.startswith('.')) and isfile(join(filepath, f))]
for file in allFiles:
    filename = join(filepath, file)
    for line in open(filename,'r'):
        if line != '\n':
            text.append(line.strip())
#             start tag is <s> and end tag is <e>
            taggedText.append('<s> ' + (line.strip()) + ' <e>')

In [4]:
import collections
wordTagDict = collections.Counter()
unigramTagDict = collections.Counter()
bigramTagDict = collections.Counter()
for line in taggedText:
    words=line.split(' ')
    for wordIndex in range(1,len(words)):
        previousWordTag = words[wordIndex-1]
        currentWordTag = words[wordIndex]
        previousWordTagArray = previousWordTag.split('/')
        currentWordTagArray = currentWordTag.split('/')
        previousWordTagArrayLength = len(previousWordTagArray)
        currentWordTagArrayLength = len(currentWordTagArray)            
        previousTag = previousWordTagArray[-1]
        currentTag = currentWordTagArray[-1]
#         any two words in between the line
        if previousWordTagArrayLength > 1 and currentWordTagArrayLength >1:
            wordTagDict[currentWordTag] +=1
            unigramTagDict[currentTag] += 1
            bigramTagDict[previousTag + ' ' + currentTag ] += 1
#        previous word is start of line
        elif previousWordTagArrayLength == 1 and currentWordTagArrayLength>1:
            wordTagDict[currentWordTag] +=1
            unigramTagDict[currentTag] += 1
            unigramTagDict[previousWordTag] += 1
            bigramTagDict[ previousWordTag + ' ' + currentTag ] += 1
#         current word is end of line
        elif previousWordTagArrayLength > 1 and currentWordTagArrayLength == 1:
            bigramTagDict[previousTag + ' ' + currentWordTag ] += 1 

In [5]:
transitionProb = {}
K=0.1
for tagPair in bigramTagDict:
    prevTag = tagPair.split(' ')[0]
    transitionProb[tagPair] = (bigramTagDict[tagPair]+K)/(unigramTagDict[prevTag] + (K*len(bigramTagDict)))

In [6]:
emissionProb = {}
K = 0.1
for wordTag in wordTagDict:
    tag = wordTag.split('/')[-1]
    emissionProb[wordTag] = (wordTagDict[wordTag] + K)/(unigramTagDict[tag] + (K * len(wordTagDict)))

In [7]:
testFileName = 'humor.txt'
testFile =  open(testFileName,'r')
testText = testFile.read()

In [8]:
updatedText = ''
for ch in range(0,len(testText)):
    updatedText += testText[ch]
    if testText[ch] == '.':
        updatedText += '\n'
    if ch > 2 and (testText[ch] == '!' and testText[ch-2] == '!') or (testText[ch] == '?' and testText[ch-2] == '?' and
                                                                      testText[ch+2] != ')'):
        updatedText += '\n'

In [9]:
testSentences = updatedText.split('\n')

In [8]:
import pandas as pd
len(testSentences)

1057

In [10]:
def findTagsPerSentence(sentence):
    index = sentence.split()
    columns = list(unigramTagDict.keys())
    df = pd.DataFrame(index = index, columns = columns)
    bp = pd.DataFrame(index = index, columns = columns)
    transitionVocab = len(bigramTagDict)
    emissionVocab = len(wordTagDict)
    for col in columns: 
        word = index[0]
#         smoothing which returns a basic value in case of unknown words
        basicTransitionProb = K / (unigramTagDict[col] + (K * transitionVocab))
        basicEmissionProb = K / (wordTagDict[word] + (K * emissionVocab))
        df.at[index[0],col] = transitionProb.get('<s> ' + col, basicTransitionProb) * emissionProb.get(word+'/'+col,basicEmissionProb)
    for i in range(1,len(index)):
        maxVal = 0.0
        maxarg = float('NaN')
        word = index[i]
        for col in columns:
            for colIndex in range(0,len(columns)):
                basicTransitionProb = K / (unigramTagDict[colIndex] + (K * transitionVocab))
                basicEmissionProb = K / (wordTagDict[word] + (K * emissionVocab))
                previousTag = df.iloc[i-1,colIndex]
                transProb = transitionProb.get(columns[colIndex] + ' ' + col, basicTransitionProb)
                emisProb = emissionProb.get(word +'/'+col, basicEmissionProb)
                checkMaxVal = previousTag * transProb * emisProb
                if checkMaxVal == maxVal and maxarg is not float('NaN'):
#                     selecting the most frequent tag in case of tie
                    if unigramTagDict[maxarg] < unigramTagDict[columns[colIndex]]:
                        maxarg = columns[colIndex]                   
                if checkMaxVal > maxVal:
                    maxVal = checkMaxVal
                    maxarg = columns[colIndex]
            df.at[index[i],col] = maxVal
            bp.at[index[i],col] = maxarg
    return df,bp

In [10]:
#  calculating tags from bp dict
def generateTaggedSentence(df,bp,i):
    tagDict = {}
    taggedSentence = ' <EOS> \n'
    firstSentence = '<sentence ID='+ str(i+1) +'> \n' 
#     finalSentence = ''
    indx = list(df.index)
    columns = list(df.columns)
    maxVal = 0.0
    maxArg = float('NaN')
    lastWord = indx[-1]
    lastWordIndex = len(indx)-1
    for colIndex in range(0,len(columns)):
        checkMaxVal = df.iloc[lastWordIndex,colIndex]
        if checkMaxVal == maxVal and maxArg is not float('NaN'):
    #                     selecting the most frequent tag in case of tie
            if unigramTagDict[maxArg] < unigramTagDict[columns[colIndex]]:
                maxArg = columns[colIndex]                   
        if checkMaxVal > maxVal:
            maxVal = checkMaxVal
            maxArg = columns[colIndex]
    lastArg = maxArg
    taggedSentence = lastWord + ', ' + maxArg + '\n' + taggedSentence
    indexList = list(range(0,len(indx),1))
    for j in indexList[::-1]:
        if j is not len(indx)-1:
            curArg = bp.iloc[j+1,columns.index(lastArg)]
            sentence = indx[j] + ', ' + curArg + ' \n '
            taggedSentence = sentence + taggedSentence
            lastArg = curArg
    taggedSentence = firstSentence + taggedSentence
    return taggedSentence   

In [13]:
#  viterbi algorithm
for i in range(92,len(testSentences)):
    print('on sentence number '+ str(i))
    df,bp = findTagsPerSentence(testSentences[i])
    taggedSentence = generateTaggedSentence(df,bp,i)
    f = open('viterbi_after92.txt','a')
    f.write(taggedSentence)
    f.close()

on sentence number 92
on sentence number 93
on sentence number 94
on sentence number 95
on sentence number 96
on sentence number 97
on sentence number 98
on sentence number 99
on sentence number 100


AttributeError: 'float' object has no attribute 'ndim'

In [19]:
def findTagsPerSentenceUsingList(sentence):
    index = sentence.split()
    columns = list(unigramTagDict.keys())
    # Creates a list containing 5 lists, each of 8 items, all set to 0
    colCount, rowCount = len(columns), len(index)
    df = [[0 for x in range(colCount)] for y in range(rowCount)]
    bp = [[0 for x in range(colCount)] for y in range(rowCount)]
    
# #     df = pd.DataFrame(index = index, columns = columns)
# #     bp = pd.DataFrame(index = index, columns = columns)
    transitionVocab = len(bigramTagDict)
    emissionVocab = len(wordTagDict)

    for col in columns: 
        word = index[0]
#         smoothing which returns a basic value in case of unknown words
        basicTransitionProb = K / (unigramTagDict[col] + (K * transitionVocab))
        basicEmissionProb = K / (wordTagDict[word] + (K * emissionVocab))
        df[0][columns.index(col)] = transitionProb.get('<s> ' + col, basicTransitionProb) * emissionProb.get(word+'/'+col,basicEmissionProb)
#     return df
    for i in range(1,len(index)):
        maxVal = 0.0
        maxarg = 0
        word = index[i]
        for col in columns:
            for colIndex in range(0,len(columns)):
                basicTransitionProb = K / (unigramTagDict[colIndex] + (K * transitionVocab))
                basicEmissionProb = K / (wordTagDict[word] + (K * emissionVocab))
                previousTag = df[i-1][colIndex]
                transProb = transitionProb.get(columns[colIndex] + ' ' + col, basicTransitionProb)
                emisProb = emissionProb.get(word +'/'+col, basicEmissionProb)
                checkMaxVal = previousTag * transProb * emisProb
#                 if checkMaxVal == maxVal and maxarg is not float('NaN'):
# #                     selecting the most frequent tag in case of tie
#                     if unigramTagDict[maxarg] < unigramTagDict[columns[colIndex]]:
#                         maxarg = colIndex
                if checkMaxVal > maxVal:
                    maxVal = checkMaxVal
                    maxarg = colIndex
            df[i][columns.index(col)] = maxVal
            bp[i][columns.index(col)] = maxarg
    return df,bp, index, columns

In [20]:
#  calculating tags from bp dict
def generateTaggedSentenceUsingList(df,bp, index, columns,i):
    tagDict = {}
    taggedSentence = ' <EOS> \n'
    firstSentence = '<sentence ID='+ str(i+1) +'> \n' 
#     finalSentence = ''
    indx = index
#     columns = list(df.columns)
    maxVal = 0.0
    maxArg = 0
    lastWord = indx[-1]
    lastWordIndex = len(indx)-1
    for colIndex in range(0,len(columns)):
        checkMaxVal = df[lastWordIndex][colIndex]                  
        if checkMaxVal > maxVal:
            maxVal = checkMaxVal
            maxArg = colIndex
    lastArg = maxArg
    taggedSentence = lastWord + ', ' + columns[maxArg] + '\n' + taggedSentence
    indexList = list(range(0,len(indx),1))
    for j in indexList[::-1]:
        if j is not len(indx)-1:
            curArg = bp[j+1][lastArg]
            sentence = indx[j] + ', ' + columns[curArg] + ' \n '
            taggedSentence = sentence + taggedSentence
            lastArg = curArg
    taggedSentence = firstSentence + taggedSentence
    return taggedSentence

In [23]:
df,bp,index,columns = findTagsPerSentenceUsingList(testSentences[18])
taggedSentence = generateTaggedSentenceUsingList(df,bp,index, columns, 18)

In [24]:
taggedSentence

'<sentence ID=19> \nWelch, pps \n wanted, qlp \n to, to \n know, dt+bez \n why, ) \n ., .\n <EOS> \n'

In [13]:
d = {}
d['a'] = 3
d['b'] = 6

In [14]:
max(d.values())

6

In [15]:
sorted(d, )

['a', 'b']

In [17]:
max(unigramTagDict.values())

149903

In [18]:
max(unigramTagDict.iteritems(), key=operator.itemgetter(1))[0]

AttributeError: 'Counter' object has no attribute 'iteritems'

In [25]:
#  viterbi algorithm
for i in range(70,len(testSentences)):
    print('Started sentence '+ str(i))
    df,bp, index, columns = findTagsPerSentenceUsingList(testSentences[i])
    taggedSentence = generateTaggedSentenceUsingList(df,bp, index, columns, i)
    f = open('viterbi_after70.txt','a')
    f.write(taggedSentence)
    f.close()

Started sentence 70
Started sentence 71
Started sentence 72
Started sentence 73
Started sentence 74
Started sentence 75
Started sentence 76
Started sentence 77
Started sentence 78
Started sentence 79
Started sentence 80
Started sentence 81


KeyboardInterrupt: 

In [26]:
len(testSentences)

1057